In [361]:
import pandas as pd             
#import matplotlib.pyplot as plt 
import datetime as dt
import numpy as np

import requests, io             # internet and input tools  
import os                       # operating system tools (check files)

from census import Census

import geopandas as gpd # this is the main geopandas 
from shapely.geometry import Polygon # also needed

#Imports below are for interactive map creation
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)

from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer, all_palettes
from bokeh.plotting import figure
from bokeh.models import Title
from bokeh.layouts import gridplot

from bokeh.plotting import figure, save
from bokeh.models import Panel, Tabs

from bokeh.resources import CDN
from bokeh.embed import file_html

from bokeh.io import output_file, show
from bokeh.models import Div

## Interactive Covid-19 map by state  

---

### Step \#1. Grab data from https://github.com/camckenzie/COVID-19-Interactive-Map



In [362]:
cases_col = ['State/Territory',
        'Total Cases', 'Cases in Last 7 Days',
        'Case Rate per 100000', 'Total Deaths']

df = pd.read_csv("https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/covid19_cases.csv",
                 skiprows=2, usecols=cases_col)

In [363]:
df

,State/Territory,Total Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths
0,Alaska,63245,1084,8645,329
1,Alabama,522131,2393,10649,10790
2,Arkansas,333407,1354,11048,5693
3,American Samoa,0,0,0,0
4,Arizona,853050,4029,11720,17151
5,California,3614112,18888,9147,59690
6,Colorado,489028,11585,8492,6188
7,Connecticut,329062,7476,9230,7995
8,District of Columbia,46579,817,6600,1095
9,Delaware,100777,2574,10349,1602


In [364]:
#Shouldn't we combine New York and NY State?
state_names = ["Alaska", "Alabama", "Arkansas", "Arizona",
               "California", "Colorado", "Connecticut", "Delaware",
               "Florida", "Georgia", "Hawaii", "Iowa", "Idaho",
               "Illinois", "Indiana", "Kansas", "Kentucky",
               "Louisiana", "Massachusetts", "Maryland", "Maine",
               "Michigan", "Minnesota", "Missouri", "Mississippi",
               "Montana", "North Carolina", "North Dakota",
               "Nebraska", "New Hampshire", "New Jersey",
               "New Mexico", "Nevada", "New York", "New York State",
               "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
               "Rhode Island", "South Carolina", "South Dakota",
               "Tennessee", "Texas", "Utah", "Virginia", "Vermont",
               "Washington", "Wisconsin", "West Virginia", "Wyoming"]

#Limits DF to only those in state_name list
df = df[df["State/Territory"].isin(state_names)]
#We need to reset the index after removing other states
#Drop = True: Do not try to insert index into dataframe columns.
df = df.reset_index(drop=True)

#total_cases = df['Total Cases'].sum()
df

,State/Territory,Total Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths
0,Alaska,63245,1084,8645,329
1,Alabama,522131,2393,10649,10790
2,Arkansas,333407,1354,11048,5693
3,Arizona,853050,4029,11720,17151
4,California,3614112,18888,9147,59690
5,Colorado,489028,11585,8492,6188
6,Connecticut,329062,7476,9230,7995
7,Delaware,100777,2574,10349,1602
8,Florida,2121387,42603,9877,34404
9,Georgia,1083300,10600,10203,19757


In [365]:
#Repeat the same process for vaccine date

vac_col = ['State/Territory/Federal Entity',
        'Total Doses Delivered', 'Doses Delivered per 100K',
        'Total Doses Administered by State where Administered',
        'Doses Administered per 100k by State where Administered']

df_vac = pd.read_csv("https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/covid19_vaccinations.csv",
                     skiprows=2, usecols=vac_col)

In [366]:
#Rename columns to fit in bar graphs later
df_vac.rename(columns={'State/Territory/Federal Entity' : "State/Territory",
                    'Total Doses Administered by State where Administered' :
                    'Total Doses Administered',
                    'Doses Administered per 100k by State where Administered' : 
                    'Doses Administered per 100k'}, inplace=True)

df_vac = df_vac[df_vac["State/Territory"].isin(state_names)]
df_vac = df_vac.reset_index(drop=True)

#total_vaccines = df1["Total Doses Delivered"].sum()
df_vac

,State/Territory,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k
0,Alaska,746705,102072,517521,70744
1,Alabama,3664740,74742,2297563,46859
2,Arkansas,2370830,78561,1618371,53627
3,Arizona,5738495,78839,4550226,62514
4,California,32618010,82552,26435308,66904
5,Colorado,4515250,78407,3846376,66792
6,Connecticut,3395165,95228,2808703,78779
7,Delaware,818475,84053,657111,67482
8,Florida,17807745,82913,13400013,62390
9,Georgia,8033225,75661,5542994,52207


In [367]:
#total_vaccines

In [368]:
#Merge the vaccination data into the main dataframe
df = df.join(df_vac["Total Doses Delivered"])
df = df.join(df_vac["Doses Delivered per 100K"])
df = df.join(df_vac["Total Doses Administered"])
df = df.join(df_vac["Doses Administered per 100k"])

#These data in these columns need to be converted to int. Currently str type
df[["Total Cases", 'Cases in Last 7 Days', 'Case Rate per 100000',
    'Total Deaths']] = df[["Total Cases", 'Cases in Last 7 Days',
                           'Case Rate per 100000', 'Total Deaths']].astype(int)

df[["Total Doses Delivered", "Doses Delivered per 100K",
    "Total Doses Administered",
    "Doses Administered per 100k"]] = df[["Total Doses Delivered",
                                          "Doses Delivered per 100K",
                                          "Total Doses Administered",
                                          "Doses Administered per 100k"]].astype(int)

df

,State/Territory,Total Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k
0,Alaska,63245,1084,8645,329,746705,102072,517521,70744
1,Alabama,522131,2393,10649,10790,3664740,74742,2297563,46859
2,Arkansas,333407,1354,11048,5693,2370830,78561,1618371,53627
3,Arizona,853050,4029,11720,17151,5738495,78839,4550226,62514
4,California,3614112,18888,9147,59690,32618010,82552,26435308,66904
5,Colorado,489028,11585,8492,6188,4515250,78407,3846376,66792
6,Connecticut,329062,7476,9230,7995,3395165,95228,2808703,78779
7,Delaware,100777,2574,10349,1602,818475,84053,657111,67482
8,Florida,2121387,42603,9877,34404,17807745,82913,13400013,62390
9,Georgia,1083300,10600,10203,19757,8033225,75661,5542994,52207


In [369]:
file_path = os.getcwd()

# local paths
doc_paths = file_path + "//docs//Cases_PNGs//"
doc_pathv = file_path + "//docs//Vaccinations_PNGs//"

html_path = "https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/docs/Cases_PNGs/"

df["cases_file_location"] = html_path + df['State/Territory'] + "_Cases.png"

html_path = "https://raw.githubusercontent.com/camckenzie/COVID-19-Interactive-Map/main/docs/Vaccinations_PNGs/"

df["vaccine_file_location"] = html_path + df['State/Territory'] + "_Vaccinations.png" 

In [370]:
df.head()

,State/Territory,Total Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k,cases_file_location,vaccine_file_location
0,Alaska,63245,1084,8645,329,746705,102072,517521,70744,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...
1,Alabama,522131,2393,10649,10790,3664740,74742,2297563,46859,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...
2,Arkansas,333407,1354,11048,5693,2370830,78561,1618371,53627,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...
3,Arizona,853050,4029,11720,17151,5738495,78839,4550226,62514,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...
4,California,3614112,18888,9147,59690,32618010,82552,26435308,66904,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...


In [371]:
#The spatial reference every created map will follow
epsg = "epsg:2163"

In [372]:
cwd = os.getcwd()

lake_shapes = cwd + "//shapefiles//lake//ne_10m_lakes.shx"

lake_map = gpd.read_file(lake_shapes)

#This changes the lake map to follow the same map projection
lake_map = lake_map.to_crs({'init': epsg})

C:\Users\Chris\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [373]:
#Gives more detail to state shapes
cwd = os.getcwd()

land_shapes = cwd + "//shapefiles//land//ne_50m_land.shx"

land_map = gpd.read_file(county_shape)

land_map = land_map.to_crs({'init': epsg})

#land_map = land_map.iloc[0:1200]

In [374]:
#Repeat the same process to create map of states
state_shapes = cwd + "//shapefiles//state//tl_2017_us_state.shx"
us_map = gpd.read_file(state_shapes)
us_map = us_map.to_crs({'init': epsg})
us_map

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62265662566,489840834,+38.6472854,-080.6183274,"POLYGON ((1561571.961 -483477.971, 1561619.813..."
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138911437206,31398800291,+28.4574302,-082.4091478,"MULTIPOLYGON (((1734277.712 -2086763.305, 1734..."
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143784114293,6211277447,+40.1028754,-089.1526108,"POLYGON ((742564.486 -445238.183, 743359.700 -..."
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206229176104,18944967530,+46.3158148,-094.1996628,"POLYGON ((245479.802 186204.457, 245483.060 18..."
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25150696145,6980371026,+38.9466584,-076.6744939,"POLYGON ((1914704.978 -379436.546, 1914710.426..."
5,1,1,44,01219835,44,RI,Rhode Island,00,G4000,A,2677997539,1323452846,+41.5974187,-071.5272723,"MULTIPOLYGON (((2314078.132 -21156.868, 231391..."
6,4,8,16,01779783,16,ID,Idaho,00,G4000,A,214048160737,2393355752,+44.3484222,-114.5588538,"POLYGON ((-1320390.644 120880.552, -1320386.43..."
7,1,1,33,01779794,33,NH,New Hampshire,00,G4000,A,23187445452,1028643155,+43.6726907,-071.5843145,"POLYGON ((2174741.054 220735.669, 2174745.020 ..."
8,3,5,37,01027616,37,NC,North Carolina,00,G4000,A,125919712692,13470113896,+35.5397100,-079.1308636,"POLYGON ((1570933.077 -826582.735, 1570940.814..."
9,1,1,50,01779802,50,VT,Vermont,00,G4000,A,23873457570,1031134839,+44.0604795,-072.6733274,"POLYGON ((2077303.749 265450.633, 2077297.944 ..."


In [375]:
#Reduces line size and shortens load time
us_map["geometry"] = us_map["geometry"].simplify(2000)

In [376]:
#Maybe we don't need because we have STATEFP already?
us_map["area_fips"] = (us_map.STATEFP.astype(str)).astype(int)

In [377]:
us_map["area_fips"]

0     54
1     12
2     17
3     27
4     24
5     44
6     16
7     33
8     37
9     50
10     9
11    10
12    35
13     6
14    34
15    55
16    41
17    31
18    42
19    53
20    22
21    13
22     1
23    49
24    39
25    48
26     8
27    45
28    40
29    47
30    56
31    15
32    38
33    21
34    78
35    69
36    66
37    23
38    36
39    32
40     2
41    60
42    26
43     5
44    28
45    29
46    30
47    20
48    18
49    72
50    46
51    25
52    51
53    11
54    19
55     4
Name: area_fips, dtype: int32

---

###  Step \#2 Merge and Clean

The next couple of cells download the requisite shapefiles from the US census. They are unzipped in a folder called shapefiles and then county. So they are assuming some structure behind your folder setup. 

In [378]:
us_map.head()

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,area_fips
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,62265662566,489840834,+38.6472854,-080.6183274,"POLYGON ((1561571.961 -483477.971, 1558865.413...",54
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,138911437206,31398800291,+28.4574302,-082.4091478,"MULTIPOLYGON (((1734277.712 -2086763.305, 1741...",12
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,143784114293,6211277447,+40.1028754,-089.1526108,"POLYGON ((742564.486 -445238.183, 747568.711 -...",17
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,206229176104,18944967530,+46.3158148,-094.1996628,"POLYGON ((245479.802 186204.457, 243282.196 20...",27
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,25150696145,6980371026,+38.9466584,-076.6744939,"POLYGON ((1914704.978 -379436.546, 1903402.221...",24


In [379]:
#Merge df data into us_map
us_map = us_map.merge(df, left_on='NAME', right_on='State/Territory', how="left", indicator=True)

In [380]:
us_map

,REGION,DIVISION,STATEFP,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,...,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k,cases_file_location,vaccine_file_location,_merge
0,3,5,54,01779805,54,WV,West Virginia,00,G4000,A,...,2763.0,8287.0,2780.0,1522165.0,84935.0,1084664.0,60523.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
1,3,5,12,00294478,12,FL,Florida,00,G4000,A,...,42603.0,9877.0,34404.0,17807745.0,82913.0,13400013.0,62390.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
2,2,3,17,01779784,17,IL,Illinois,00,G4000,A,...,22745.0,10256.0,23945.0,10246165.0,80858.0,8306266.0,65549.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
3,2,4,27,00662849,27,MN,Minnesota,00,G4000,A,...,14259.0,9833.0,7083.0,4194140.0,74369.0,3787907.0,67166.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
4,3,5,24,01714934,24,MD,Maryland,00,G4000,A,...,9298.0,7212.0,8562.0,4967520.0,82166.0,4110450.0,67990.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
5,1,1,44,01219835,44,RI,Rhode Island,00,G4000,A,...,2633.0,13607.0,2647.0,895785.0,84559.0,787717.0,74358.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
6,4,8,16,01779783,16,ID,Idaho,00,G4000,A,...,1395.0,10339.0,2017.0,1283065.0,71797.0,981273.0,54910.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
7,1,1,33,01779794,33,NH,New Hampshire,00,G4000,A,...,2833.0,6713.0,1266.0,1096375.0,80633.0,1102415.0,81077.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
8,3,5,37,01027616,37,NC,North Carolina,00,G4000,A,...,14287.0,8998.0,12387.0,8309510.0,79228.0,6300491.0,60073.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
9,1,1,50,01779802,50,VT,Vermont,00,G4000,A,...,904.0,3505.0,242.0,570360.0,91405.0,482794.0,77372.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both


In [381]:
#STATEFP is given ID for each state
us_map.set_index("STATEFP", inplace = True)

#This removes all territory data (Puerto Rico, Guam, Virgin Islands, etc)
drop_list = ["72","78","69","66","60","11"]
us_map.drop(drop_list, inplace = True)

us_map.reset_index()
us_map.head()

,REGION,DIVISION,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,...,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Total Doses Delivered,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k,cases_file_location,vaccine_file_location,_merge
STATEFP,,,,,,,,,,,,,,,,,,,,,
54,3,5,01779805,54,WV,West Virginia,00,G4000,A,62265662566,...,2763.0,8287.0,2780.0,1522165.0,84935.0,1084664.0,60523.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
12,3,5,00294478,12,FL,Florida,00,G4000,A,138911437206,...,42603.0,9877.0,34404.0,17807745.0,82913.0,13400013.0,62390.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
17,2,3,01779784,17,IL,Illinois,00,G4000,A,143784114293,...,22745.0,10256.0,23945.0,10246165.0,80858.0,8306266.0,65549.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
27,2,4,00662849,27,MN,Minnesota,00,G4000,A,206229176104,...,14259.0,9833.0,7083.0,4194140.0,74369.0,3787907.0,67166.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both
24,3,5,01714934,24,MD,Maryland,00,G4000,A,25150696145,...,9298.0,7212.0,8562.0,4967520.0,82166.0,4110450.0,67990.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both


In [382]:
#Create labels for the map pngs and format numbers >999
#Cases Map
us_map["cases_label"] = us_map["Total Cases"].round(0)
us_map["cases_label"] = us_map["cases_label"].map('{:,.0f}'.format)
us_map["death_label"] = us_map["Total Deaths"].round(0)
us_map["death_label"] = us_map["death_label"].map('{:,.0f}'.format)

#Vaccine map
us_map["delivered_label"] = us_map["Total Doses Delivered"].round(0)
us_map["delivered_label"] = us_map["delivered_label"].map('{:,.0f}'.format)
us_map["administered_label"] = us_map["Total Doses Administered"].round(0)
us_map["administered_label"] = us_map["administered_label"].map('{:,.0f}'.format)

us_map.head()

,REGION,DIVISION,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,...,Doses Delivered per 100K,Total Doses Administered,Doses Administered per 100k,cases_file_location,vaccine_file_location,_merge,cases_label,death_label,delivered_label,administered_label
STATEFP,,,,,,,,,,,,,,,,,,,,,
54,3,5,01779805,54,WV,West Virginia,00,G4000,A,62265662566,...,84935.0,1084664.0,60523.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"148,517","2,780","1,522,165","1,084,664"
12,3,5,00294478,12,FL,Florida,00,G4000,A,138911437206,...,82913.0,13400013.0,62390.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"2,121,387","34,404","17,807,745","13,400,013"
17,2,3,01779784,17,IL,Illinois,00,G4000,A,143784114293,...,80858.0,8306266.0,65549.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"1,299,575","23,945","10,246,165","8,306,266"
27,2,4,00662849,27,MN,Minnesota,00,G4000,A,206229176104,...,74369.0,3787907.0,67166.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"554,536","7,083","4,194,140","3,787,907"
24,3,5,01714934,24,MD,Maryland,00,G4000,A,25150696145,...,82166.0,4110450.0,67990.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"436,028","8,562","4,967,520","4,110,450"


In [383]:
q_cases = [0, 10000, 25000, 50000, 100000, 250000, 500000, 750000, 1000000, 2000000, np.inf]
q_vaccines = [400000,800000,1000000,2000000,3000000,4000000,5000000,10000000,15000000,30000000,np.inf]

us_map["q_cases"]= pd.cut(us_map["Total Cases"], q_cases, labels=range(0,10))
us_map["q_vaccines"]= pd.cut(us_map["Total Doses Delivered"], q_vaccines, labels=range(0,10))

#Replaceing N/As
us_map["q_cases"].fillna(0, inplace = True)
us_map["q_vaccines"].fillna(0, inplace = True)

us_map["cases_label"].replace("nan", "N.R.", inplace=True)
us_map["death_label"].replace("nan", "N.R.", inplace=True)

us_map["delivered_label"].replace("nan", "N.R.", inplace=True)
us_map["administered_label"].replace("nan", "N.R.", inplace=True)

us_map.head()

,REGION,DIVISION,STATENS,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,...,Doses Administered per 100k,cases_file_location,vaccine_file_location,_merge,cases_label,death_label,delivered_label,administered_label,q_cases,q_vaccines
STATEFP,,,,,,,,,,,,,,,,,,,,,
54,3,5,01779805,54,WV,West Virginia,00,G4000,A,62265662566,...,60523.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"148,517","2,780","1,522,165","1,084,664",4,2
12,3,5,00294478,12,FL,Florida,00,G4000,A,138911437206,...,62390.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"2,121,387","34,404","17,807,745","13,400,013",9,8
17,2,3,01779784,17,IL,Illinois,00,G4000,A,143784114293,...,65549.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"1,299,575","23,945","10,246,165","8,306,266",8,7
27,2,4,00662849,27,MN,Minnesota,00,G4000,A,206229176104,...,67166.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"554,536","7,083","4,194,140","3,787,907",6,5
24,3,5,01714934,24,MD,Maryland,00,G4000,A,25150696145,...,67990.0,https://raw.githubusercontent.com/camckenzie/C...,https://raw.githubusercontent.com/camckenzie/C...,both,"436,028","8,562","4,967,520","4,110,450",5,5


In [384]:
us_map = gpd.overlay(us_map, land_map, how='intersection')

In [385]:
great_lakes = ["Lake Superior", "Lake Michigan", "Lake Erie", "Lake Superior", "Lake Huron"]
us_map = gpd.overlay(us_map, lake_map[lake_map.name.isin(great_lakes)], how='difference')

# Step3:creating live interactive html page 

In [386]:
state_geosource = GeoJSONDataSource(geojson = us_map.to_json())

palette = all_palettes['Cividis'][11]
#reversed_palette = palette[::-1]

color_mapper_case = LinearColorMapper(palette = palette, low=0, high=9)
color_mapper_vac = LinearColorMapper(palette = palette, low=0, high=9)
#tick_labels = {2:str(q_cases[1]), 4:str(q_cases[4]), 6:str(q_cases[6]), 8:str(q_cases[9])}
case_ticks = {2:str(q_cases[1]), 4:str(q_cases[4]), 6:str(q_cases[6]), 8:str(q_cases[9])}
vac_ticks = {2:str(q_vaccines[1]), 4:str(q_vaccines[4]), 6:str(q_vaccines[6]), 8:str(q_vaccines[9])}

today = dt.date.today()
d = today.strftime("%B %d, %Y")

In [387]:
title = "COVID-19 Cases by State as of " + d + " || Total Cases: " + f"{int(us_map['Total Cases'].sum()):,d}"

# Create figure object.
p = figure( 
           plot_height = 800,
           plot_width = 1300, 
           toolbar_location = None)

p.toolbar.active_drag = None
#p.toolbar.active_scroll = None
#p.toolbar.active_tap = None

def interactive_map_tab_cases():
    color_bar = ColorBar(color_mapper = color_mapper_case, 
                     label_standoff = 8,
                     width = 20, height = 420,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0), major_label_overrides = case_ticks,
                     major_label_text_align="left",
                     major_tick_line_alpha = .25)

    descip = "Data from https://covid.cdc.gov/covid-data-tracker/#cases_totalcases"
    p.add_layout(Title(text=descip, text_font_style="italic", text_font_size="9pt"), 'above')
    p.add_layout(Title(text=title, text_font_size="11pt"), 'above')
    color_bar.background_fill_color = "#F5F5F5"
    p.add_layout(color_bar, "right")

    

    
def interactive_map_layout_cases():
    author = "Created by Team Coach"
    p.add_layout(Title(text=author, text_font_style="italic", text_font_size="9pt"), 'below')

    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    p.background_fill_color = "grey"
    p.background_fill_alpha = 0.25
    p.border_fill_color = "#F5F5F5"
    p.toolbar.autohide = True
   


states = p.patches('xs','ys', source = state_geosource,
                   fill_color = {"field" :'q_cases',
                                 "transform" : color_mapper_case},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

state_line = p.multi_line('xs','ys', source = state_geosource,
                   line_color = "black", 
                   line_width = 0.25)


#create hover tool

def create_hover_tool_cases():
    TOOLTIPS = """
        <div style="background-color:#F5F5F5; opacity: 0.95;">
            <div style = "text-align:center;">
                <span style="font-size: 12px; font-weight: bold;">@NAME
        </div>
        <div>
            <img
                src="@cases_file_location" height="375" alt="@cases_file_location" width="450"
                style="float: center; margin: 1px 1px 1px 1px; opacity: 0.95;"
                border="0"
            ></img>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold">Cases: @cases_label &nbsp &nbsp Deaths: @death_label &nbsp &nbsp 
            </span>
        </div>
        </div>
        """
    p.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))

In [388]:
interactive_map_tab_cases()
interactive_map_layout_cases()
create_hover_tool_cases()

In [389]:
title_vaccines = "COVID-19 Vaccinations by State as of " + d + " || Total Doses Delivered: " + f"{int(us_map['Total Doses Delivered'].sum()):,d}"
# Create figure object.
pvac = figure( 
           plot_height = 600,
           plot_width = 950, 
           toolbar_location = None)

pvac.toolbar.active_drag = None
#pvac.toolbar.active_scroll = None
#pvac.toolbar.active_tap = None

def interactive_map_tab_vac():
    color_bar = ColorBar(color_mapper = color_mapper_vac, 
                     label_standoff = 8,
                     width = 20, height = 420,
                     border_line_color = None,
                     orientation = "vertical",
                     location=(0,0), major_label_overrides = vac_ticks,
                     major_label_text_align= "left",
                     major_tick_line_alpha = .25)


    descip = "Data from https://covid.cdc.gov/covid-data-tracker/#vaccinations"
    pvac.add_layout(Title(text=descip, text_font_style="italic", text_font_size="9pt"), 'above')
    pvac.add_layout(Title(text=title_vaccines, text_font_size="11pt"), 'above')
    color_bar.background_fill_color = "#F5F5F5"
    pvac.toolbar.autohide = True
    pvac.add_layout(color_bar, "right")


def interactive_map_layout_vac():
    author = "Created by Team Coach"
    pvac.add_layout(Title(text=author, text_font_style="italic", text_font_size="9pt"), 'below')

    pvac.xgrid.grid_line_color = None
    pvac.ygrid.grid_line_color = None
    pvac.axis.visible = False
    pvac.background_fill_color = "grey"
    pvac.background_fill_alpha = 0.25
    pvac.border_fill_color = "#F5F5F5"


# Add patch renderer to figure.

states = pvac.patches('xs','ys', source = state_geosource,
                   fill_color = {"field" :'q_vaccines',
                                 "transform" : color_mapper_vac},
                   line_color = "gray", 
                   line_width = 0.25, 
                   fill_alpha = 1)

state_line = pvac.multi_line('xs','ys', source = state_geosource,
                   line_color = "black", 
                   line_width = 0.25)



#create hover tool:
def create_hover_tool_vac():
    TOOLTIPS = """
        <div style="background-color:#F5F5F5; opacity: 0.95;">
            <div style = "text-align:center;">
                <span style="font-size: 12px; font-weight: bold;">@NAME
        </div>
        <div>
            <img
                src="@vaccine_file_location" height="375" alt="@vaccine_file_location" width="450"
                style="float: center; margin: 1px 1px 1px 1px; opacity: 0.95;"
                border="0"
            ></img>
        </div>
        <div style = "text-align:center;">
            <span style="font-size: 12px; font-weight: bold">Delivered: @delivered_label &nbsp &nbsp Administered: @administered_label</span>
        </div>
        </div>
        """

    pvac.add_tools(HoverTool(renderers = [states],
                      tooltips = TOOLTIPS))

In [390]:
interactive_map_tab_vac()
interactive_map_layout_vac()
create_hover_tool_vac()

In [391]:
def tab_creation_saving():
    tab1 = Panel(child=p, title="Cases")
    tab2 = Panel(child=pvac, title="Vaccines")

    tabs = Tabs(tabs=[ tab1, tab2 ])
    #select = Select(title="Option:", value="foo", options=["Cases", "Vaccines"])
    #select.js_on_change("value", CustomJS(code="""
    #console.log('select: value=' + this.value, this.toString())
    #"""))
    doc_path = file_path +"//Documents"
    outfp = doc_path + "//us_covid_map.html"
    print(outfp)
    save(tabs, outfp)
    #html = file_html(select, CDN, outfp)
    html = file_html(tabs, CDN, outfp)

In [392]:
tab_creation_saving()

C:\Users\Chris//Documents//us_covid_map.html


C:\Users\Chris\anaconda3\lib\site-packages\bokeh\io\saving.py:126: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\Users\Chris\anaconda3\lib\site-packages\bokeh\io\saving.py:139: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
